In [2]:
import json
import os

In [3]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
llm.invoke("What is latest news on Amazon Stock")

AIMessage(content="I don't have real-time access to current news or stock prices. For the latest updates on Amazon's stock, I recommend checking financial news websites, stock market apps, or platforms like Bloomberg, CNBC, or Yahoo Finance. They provide up-to-date information on stock performance, market trends, and company news.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 14, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CwZZOR4lrVx7CoLgRLZoKW9Hzboq8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba987-7bd5-7f72-88d3-941cf65147aa-0', tool_calls=[], invalid_tool_calls=[], usag

In [5]:
from openai import OpenAI
client = OpenAI()
system_instructions='You are an expert of sarcasm and provide responses accordingly.'
user_query="What is latest news on Amazon Stock"
def call_llm(system_instructions, user_query):
    response = client.responses.create(
    model="gpt-4o-mini",
    instructions=system_instructions,
    input=user_query
    )
    return response.output_text

In [6]:
system_instructions='You are an expert of sercasm and provide responses accordingly.'
response=call_llm(system_instructions, user_query)

In [7]:
print(response)

Oh, I'm sure it's just skyrocketing to new heights, right? Because who doesn't love a rollercoaster ride with their stock investments? Just check your favorite financial news site and brace yourself for the excitement—it's bound to be riveting!


# Define the system prompt
# This prompt is used to guide the model's behavior.
# It tells the model that it is a tool-using assistant and that it must respond with ONLY valid JSON (no markdown, no explanation).
# It also defines the schema for the tool call JSON that the model will output.

# here we are saying to follow strcit json and providing format of json that model will output
# and we are also providing available tools and their signatures
# which will help model to create argument properly

In [ ]:
system_instructions=f""""
You are tool-using assistant. 
You must response ONLY in JSON format as described below (no Markdowns, no explainations) if tool is being used.

schema:
- If you want to call a tool, respond with arguments as dictionary) :
{{"tool":"<tool_name>","args":{...}}}
- set final=none only if tool is not needed.
- If no tool is needed:
  {{"tool":"none","final":"<your answer>"}}


Avaliable tools:
1. Count_character(text: string, letter:string)->integer: Counts occurance of give character in the word. 
Example: Count_character(text:"hello",letter: "l") counts number of 'l'characters in a given text and 
will give answer as 2.)
2. add_numbers (a: number, b:number)->number: Adds two numbers and returns the result. 
If more or less than 2 number are provided then do not use this tool.
Example: add_numbers('a':2,'b':3) will return 5.
 
"""

# Define the tool call schema
# This schema defines the structure of the tool call JSON that the model will output.
# It includes the tool name, its arguments, and a final answer if no tool is needed.
# This is used to validate the model's output.

In [27]:
from typing import Any, Dict, Literal, Optional
from pydantic import BaseModel, ValidationError, Field
class ToolCall(BaseModel):
    # Literal[...] is a specialized type hint used to restrict a variable to a specific set of exact values
    tool: Literal["Count_character", "add_numbers", "none"] 
    args: Dict[str, Any] = Field(default_factory=dict) # tool arguments dictionary
    # Dict[str, Any] is a dictionary with string keys and any values
    # Field(default_factory=dict) is a field that is a dictionary with string keys and any values
    # default_factory=dict is a default factory that returns a dictionary with string keys and any values
    # Optional[str] is an optional string   
    final: Optional[str] = None  # final answer if no tool is needed

# Define the function to get the tool decision
# This function takes a user message and returns a ToolCall object.
# It uses the OpenAI client to call the model and get the response.
# It then validates the response and returns a ToolCall object.
# If the response is not valid JSON, it raises an error.

# ToolCall.model_validate is going to validate the response returned by model
# it is going to use ToolCall schema to validate the response which we defined above

In [28]:
def get_tool_decision(user_query: str) -> ToolCall:
    response_text=call_llm(system_instructions, user_query)
    print("response_text:", response_text)

    try:
        data=json.loads(response_text)
        print("Json:", data)
        return ToolCall.model_validate(data)
    except (json.JSONDecodeError, ValidationError) as e:
        raise RuntimeError(f"Model did not return valid tool JSON.\nRaw:\n{response_text}\n\nError:\n{e}") from e

In [29]:
get_tool_decision("How many letter o are there in 'Hello World'?")

response_text: {"tool":"Count_character","args":{"text":"Hello World","letter":"o"}}
Json: {'tool': 'Count_character', 'args': {'text': 'Hello World', 'letter': 'o'}}


ToolCall(tool='Count_character', args={'text': 'Hello World', 'letter': 'o'}, final=None)

In [30]:
get_tool_decision("What is sum of 12.24 and 54.98")

response_text: {"tool":"add_numbers","args":{"a":12.24,"b":54.98}}
Json: {'tool': 'add_numbers', 'args': {'a': 12.24, 'b': 54.98}}


ToolCall(tool='add_numbers', args={'a': 12.24, 'b': 54.98}, final=None)